In [1]:
import pandas as pd
import numpy as np

# set the display to 160 rows to view all data
pd.options.display.max_columns=160
pd.options.display.max_rows = 160

In [2]:
df=pd.read_csv('usa-2016-presidential-election-by-county.csv', sep=';')

In [3]:
df.head(5)

,State,ST,Fips,County,Precincts,Votes,Democrats 08 (Votes),Democrats 12 (Votes),Republicans 08 (Votes),Republicans 12 (Votes),Republicans 2016,Democrats 2016,Green 2016,Libertarians 2016,Republicans 2012,Republicans 2008,Democrats 2012,Democrats 2008,Less Than High School Diploma,At Least High School Diploma,At Least Bachelors's Degree,Graduate Degree,School Enrollment,Median Earnings 2010,White (Not Latino) Population,African American Population,Native American Population,Asian American Population,Other Race or Races,Latino Population,Children Under 6 Living in Poverty,Adults 65 and Older Living in Poverty,Total Population,Preschool.Enrollment.Ratio.enrolled.ages.3.and.4,Poverty.Rate.below.federal.poverty.threshold,Gini.Coefficient,Child.Poverty.living.in.families.below.the.poverty.line,Management.professional.and.related.occupations,Service.occupations,Sales.and.office.occupations,Farming.fishing.and.forestry.occupations,Construction.extraction.maintenance.and.repair.occupations,Production.transportation.and.material.moving.occupations,White,Black,Hispanic,Asian,Amerindian,Other,White_Asian,SIRE_homogeneity,median_age,lon,lat,Poor.physical.health.days,Poor.mental.health.days,Low.birthweight,Teen.births,Children.in.single.parent.households,Adult.smoking,Adult.obesity,Diabetes,Sexually.transmitted.infections,HIV.prevalence.rate,Uninsured,Unemployment,Violent.crime,Homicide.rate,Injury.deaths,Infant.mortality,CA,S,MAR,CFS,ACFS,MeanALC,MaxALC,Mixedness,elevation,annual_PRCP,winter_PRCP,summer_PRCP,spring_PRCP,autumn_PRCP,annual_TAVG,annual_TMAX,annual_TMIN,winter_TAVG,winter_TMAX,winter_TMIN,summer_TAVG,summer_TMAX,summer_TMIN,spring_TAVG,spring_TMAX,spring_TMIN,autumn_TAVG,autumn_TMAX,autumn_TMIN,nearest_county,temp,precip,temp_bins,lat_bins,lon_bins,precip_bins,elevation_bins,Geo Shape,name_16,reporting,votes16_trumpd,votes16_clintonh,votes16_johnsong,votes16_steinj,votes16_castled,votes16_de_la_fuenter,est_votes_remaining,votes16_mcmulline,votes16_hedgesj,votes16_kahnl,votes16_la_rivag,votes16_hoeflingt,votes16_kenistonc,votes16_smithm,votes16_atwoodf,votes16_kennedya,votes16_kopitkek,votes16_kotlikoffl,votes16_lyttleb,votes16_maldonadoj,votes16_maturenm,votes16_scottr,votes16_silvar,votes16_soltysike,votes16_vacekd,votes16_copelands,votes16_jacobp,votes16_whitej,votes16_mooreheadm,votes16_none_of_these_candidates,votes16_duncanr,votes16_skewesp,votes16_giordanir,total16,other16_frac,rep16_frac2,dem16_frac2,name_prev,statecode_prev,total08,total12,other08,other12,other12_frac,other08_frac,rep12_frac2,rep08_frac2,dem12_frac2,dem08_frac2
0,Arkansas,AR,5085,"Lonoke County, Arkansas",55.0,27007.0,5968.0,5625.0,17242.0,17880.0,73.692006,20.876069,0.629466,2.814085,74.147798,72.625416,23.326698,25.137947,14.2,85.8,16.6,4.9,75.45,28134.093970,88.0,6.35,0.50,0.75,1.45,3.00,23.65,12.95,65890,45.8,12.45,0.391,16.80,29.90,14.15,28.05,1.30,12.20,14.35,88.0,6.35,3.00,0.75,0.50,1.45,88.75,0.779624,35.3,-91.894132,34.755114,3.3,3.9,0.08,47.1,0.271,0.206,0.312,0.108,317.3,57.3,0.165,0.061,390.71,4.24,71.1,6.8,0.332542,-0.065772,0.424415,0.000006,0.000006,0.000022,0.000098,-0.955205,80.90,4963.0,1229.0,958.5,1469.0,1306.5,617.5,728.0,506.5,424.0,528.5,319.5,797.5,904.5,691.0,617.0,730.5,503.0,627.0,744.5,509.5,5085.0,16.527778,1260.602,"14.9, 16.6","34.7542703702, -91.8886676034","(-93.6,-90.4]","(1.19e+03,1.27e+03]","(38.9,105]","{""type"": ""Polygon"", ""coordinates"": [[[-92.0755...",Lonoke,55.0,19902.0,5638.0,760.0,170.0,78.0,NaN,0.0,241.0,119.0,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27007.0,0.028141,0.779248,0.220752,Lonoke,5.0,23741.0,24114.0,531.0,609.0,0.025255,0.022366,0.760689,0.742869,0.239311,0.257131
1,Arkansas,AR,5095,"Monroe County, Arkansas",16.0,2778.0,1615.0,1583.0,1754.0,1585.0,50.395968,46.724262,0.323974,1.259899,49.071207,50.855320,49.009288,46.825167,31.4,68.6,12.3,3.7,80.25,20974.931745,56.8,39.90,0.50,0.95,0.55,1.30,44.00,23.55,8377,52.0,22.65,0.470,28.80,24.20

In [4]:
for i in df.columns:
    print(i)

State
ST
Fips
County
Precincts
Votes
Democrats 08 (Votes)
Democrats 12 (Votes)
Republicans 08 (Votes)
Republicans 12 (Votes)
Republicans 2016
Democrats 2016
Green 2016
Libertarians 2016
Republicans 2012
Republicans 2008
Democrats 2012
Democrats 2008
Less Than High School Diploma
At Least High School Diploma
At Least Bachelors's Degree
Graduate Degree
School Enrollment
Median Earnings 2010
White (Not Latino) Population
African American Population
Native American Population
Asian American Population
Other Race or Races
Latino Population
Children Under 6 Living in Poverty
Adults 65 and Older Living in Poverty
Total Population
Preschool.Enrollment.Ratio.enrolled.ages.3.and.4
Poverty.Rate.below.federal.poverty.threshold
Gini.Coefficient
Child.Poverty.living.in.families.below.the.poverty.line
Management.professional.and.related.occupations
Service.occupations
Sales.and.office.occupations
Farming.fishing.and.forestry.occupations
Construction.extraction.maintenance.and.repair.occupations
Produ

In [5]:
#replace space with underscore in column names
df.columns = [column.replace(" ","_") for column in df.columns]
#replace period with underscore in column names for consistency
df.columns = [column.replace(".","_") for column in df.columns]

In [6]:
#drop unnecessary columns (duplicate or else)
def columns_to_drop(a,b):
    return list(range(a,b))

cols = columns_to_drop(43, 50) + columns_to_drop(70, 78) + columns_to_drop(80, 84) + columns_to_drop(87, 99) + columns_to_drop(102, 108) + columns_to_drop(116, 143)
df.drop(df.columns[cols], axis=1, inplace=True)

In [7]:
#review total16=null, looks like nobody casted any votes for that county in 2016. So let's drop them.
df[df.total16.isnull()]
df=df.dropna(subset=["total16"])

In [8]:
#fill in the NaN Green_2016, votes16_steinj, votes16_castled, votes16_de_la_fuenter with 0. 
df.Green_2016.fillna(0, inplace=True)
df.votes16_steinj.fillna(0,inplace=True)
df.votes16_castled.fillna(0,inplace=True)
df.votes16_de_la_fuenter.fillna(0,inplace=True)

In [9]:
#remove additional columns as they are not critical in the analysis (geography information, temperature etc)
df.drop(columns=["elevation","annual_PRCP","annual_TAVG","annual_TMAX","annual_TMIN","nearest_county",
                 "temp","precip","lon","lat"],inplace=True)

In [10]:
# write a function that will include a column with the state mean of the variable. 
def fillnull_avg(col_name_original):
    col_name_new = col_name_original+'_avg'
    df[col_name_new] = df.groupby('State')[col_name_original].transform(lambda x: np.mean(x))
    df[col_name_original].fillna(df[col_name_new], inplace=True)
    df.drop(columns=[col_name_new], inplace=True)

In [11]:
# Put in a list of columns with null values 
col_fill_list = ["Poor_physical_health_days","Poor_mental_health_days","Low_birthweight",
             "Teen_births","Children_in_single_parent_households","Adult_smoking",
             "Sexually_transmitted_infections","HIV_prevalence_rate","Violent_crime",
             "Injury_deaths","Preschool_Enrollment_Ratio_enrolled_ages_3_and_4",
                "Homicide_rate","Infant_mortality"]

In [12]:
# fill in the null values with average state values
for i in col_fill_list:
    fillnull_avg(i)

In [13]:
# HIV and Homicide Rate still have nulls, let's fill them with the nation's average. 

df["HIV_prevalence_rate"].fillna(df.HIV_prevalence_rate.mean(),inplace=True)
df["Homicide_rate"].fillna(df.HIV_prevalence_rate.mean(),inplace=True)

In [14]:
# Create a new column "Result_2016" that gives the result of election
df["Result_2016"]=df[["Republicans_2016","Democrats_2016","Green_2016","Libertarians_2016"]].idxmax(axis=1)

In [15]:
# Remove additional columns that are not needed for this analysis
cols = columns_to_drop(0, 3) + columns_to_drop(4, 18) + columns_to_drop(61, 85)
df.drop(df.columns[cols], axis=1, inplace=True)

In [16]:
# Remove parenthesis in column names
df.columns = [column.replace("(","") for column in df.columns]
df.columns = [column.replace(")","") for column in df.columns]

In [17]:
# Code the result of 2016 to a 0 (Republican) and 1 (Democrat)

df['Result_2016']= df['Result_2016'].replace(['Republicans_2016'], 0)
df['Result_2016']= df['Result_2016'].replace(['Democrats_2016'], 1)

In [18]:
df.head(5)

,County,Less_Than_High_School_Diploma,At_Least_High_School_Diploma,At_Least_Bachelors's_Degree,Graduate_Degree,School_Enrollment,Median_Earnings_2010,White_Not_Latino_Population,African_American_Population,Native_American_Population,Asian_American_Population,Other_Race_or_Races,Latino_Population,Children_Under_6_Living_in_Poverty,Adults_65_and_Older_Living_in_Poverty,Total_Population,Preschool_Enrollment_Ratio_enrolled_ages_3_and_4,Poverty_Rate_below_federal_poverty_threshold,Gini_Coefficient,Child_Poverty_living_in_families_below_the_poverty_line,Management_professional_and_related_occupations,Service_occupations,Sales_and_office_occupations,Farming_fishing_and_forestry_occupations,Construction_extraction_maintenance_and_repair_occupations,Production_transportation_and_material_moving_occupations,SIRE_homogeneity,median_age,Poor_physical_health_days,Poor_mental_health_days,Low_birthweight,Teen_births,Children_in_single_parent_households,Adult_smoking,Adult_obesity,Diabetes,Sexually_transmitted_infections,HIV_prevalence_rate,Uninsured,Unemployment,Violent_crime,Homicide_rate,Injury_deaths,Infant_mortality,Result_2016
0,"Lonoke County, Arkansas",14.2,85.8,16.6,4.9,75.45,28134.093970,88.0,6.35,0.50,0.75,1.45,3.00,23.65,12.95,65890,45.8,12.45,0.391,16.80,29.90,14.15,28.05,1.30,12.20,14.35,0.779624,35.3,3.3,3.900000,0.08,47.1,0.271,0.206,0.312,0.108,317.3,57.3,0.165,0.061,390.71,4.240000,71.1,6.800,0
1,"Monroe County, Arkansas",31.4,68.6,12.3,3.7,80.25,20974.931745,56.8,39.90,0.50,0.95,0.55,1.30,44.00,23.55,8377,52.0,22.65,0.470,28.80,24.20,18.15,23.00,5.75,6.85,22.15,0.482140,43.2,5.3,4.281429,0.12,70.7,0.530,0.368,0.351,0.141,1572.8,345.1,0.200,0.084,202.77,9.861842,112.9,8.820,0
2,"Drew County, Arkansas",19.4,80.6,19.4,5.7,78.50,21988.947300,67.9,27.95,0.15,0.55,0.85,2.65,34.15,14.40,18604,73.5,23.45,0.469,36.95,28.35,18.25,20.15,3.60,9.40,20.20,0.539968,36.8,4.5,4.700000,0.10,47.7,0.429,0.181,0.323,0.126,747.3,79.9,0.197,0.108,449.02,9.861842,82.0,8.820,0
3,"Madison County, Arkansas",24.2,75.8,13.4,4.1,76.40,23365.948890,92.0,0.25,1.25,0.30,1.50,4.70,26.20,18.45,15597,43.8,18.20,0.446,25.15,25.15,16.85,19.55,2.90,16.20,19.40,0.849006,38.5,4.9,2.100000,0.07,49.0,0.179,0.304,0.328,0.135,247.2,131.3,0.239,0.053,245.83,9.861842,96.8,8.820,0
4,"Jasper County, Georgia",21.7,78.3,13.3,5.6,70.20,25363.060950,72.7,22.20,0.40,0.10,1.05,3.55,17.60,17.95,13744,36.5,18.05,0.451,27.60,21.60,17.15,22.60,4.35,15.10,19.25,0.579201,38.1,4.3,3.700000,0.09,54.1,0.381,0.210,0.298,0.118,324.1,402.4,0.239,0.096,205.60,6.990811,71.6,9.172,0


In [20]:
df.to_csv('ElectionTable_cleandata.csv', index=False)